# UNDER CONSTRUCTION

This notebook is still a work in progress and may not work as intended. You have been warned!

# Boots 'n' Cats 2c: Modelling with a Custom MXNet Algorithm

In this notebook we'll try another approach to build our boots 'n' cats detector: a [GluonCV](https://gluon-cv.mxnet.io/)-based [YOLOv3](https://pjreddie.com/media/files/papers/YOLOv3.pdf) implementation on SageMaker's [MXNet container](https://sagemaker.readthedocs.io/en/stable/using_mxnet.html).

SageMaker supports fully custom containers, but also offers pre-optimized environments for the major ML frameworks TensorFlow, PyTorch, and MXNet; which streamline typical workflows.

The interface mechanisms (channels, endpoints, etc) work the same as for the built-in algorithms, but now we're authoring a Python package loaded by the framework application inside the base container: So need to understand the interfaces through which our code consumes inputs and exposes results and parameters.

**You'll need to** have gone through the first notebook in this series (*Intro and Data Preparation*) to complete this example. The explanatory notes here also reference the equivalent steps of the previous notebook *SageMaker Built-In Algorithm* - so you'll want to read through that if you haven't already.

## About the Algorithm: YOLOv3

As discussed with reference to benchmarks on the project [website](https://pjreddie.com/darknet/yolo/) and detailed in the [original paper](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Redmon_You_Only_Look_CVPR_2016_paper.pdf) (Redmon et al, 2016), YOLO ("You Only Look Once") is another highly successful object detection algorithm alongside SSD as implemented in the SageMaker built-in algorithm.

Both YOLO and SSD are "one-stage detectors", in contrast to previous R-CNN group methods which separately 1) propose and then 2) validate and adjust bounding boxes. Tackling the region proposal and classification/validation problems together gives these architectures significant speed benefits at comparable accuracy.

Whereas SSD creates convolutional "feature maps" at different scales and learns to predict the offset of "anchor boxes" relative to those; YOLO in parallel computes "class probabilities" on subdivided grid squares of the image, and bounding box coordinates for likely objects - before correlating the two together.

A nice comparison of the methods is presented [here](https://lilianweng.github.io/lil-log/2018/12/27/object-detection-part-4.html). Recent advances in YOLO (v2 and v3 releases) have led to it achieving better accuracy than SSD at comparable model sizes / speeds in some benchmarks, as shown in the below graph reproduced from the [GluonCV Model Zoo](https://gluon-cv.mxnet.io/model_zoo/detection.html)

<img src="BlogImages/GluonCVYOLOvsSSD.png"/>

## Step 0: Dependencies and configuration

As usual we'll start by loading libraries, defining configuration, and connecting to the AWS SDKs:

In [ ]:
%load_ext autoreload
%autoreload 1

# Built-Ins:
import csv
import os
from collections import defaultdict
import json

# External Dependencies:
import boto3
import imageio
import numpy as np
import sagemaker
from sagemaker.mxnet import MXNet as SageMakerMXNet
from IPython.display import display, HTML

# Local Dependencies:
%aimport util

Next we re-load configuration from the intro & data processing notebook:

In [ ]:
%store -r BUCKET_NAME
assert BUCKET_NAME, "BUCKET_NAME missing from IPython store"
%store -r CHECKPOINTS_PREFIX
assert CHECKPOINTS_PREFIX, "CHECKPOINTS_PREFIX missing from IPython store"
%store -r DATA_PREFIX
assert DATA_PREFIX, "DATA_PREFIX missing from IPython store"
%store -r MODELS_PREFIX
assert MODELS_PREFIX, "MODELS_PREFIX missing from IPython store"
%store -r CLASS_NAMES
assert CLASS_NAMES, "CLASS_NAMES missing from IPython store"
%store -r test_image_folder
assert test_image_folder, "test_image_folder missing from IPython store"

%store -r attribute_names
assert attribute_names, "attribute_names missing from IPython store"
%store -r n_samples_training
assert n_samples_training, "n_samples_training missing from IPython store"
%store -r n_samples_validation
assert n_samples_validation, "n_samples_validation missing from IPython store"

Here we just connect to the AWS SDKs we'll use, and validate the choice of S3 bucket:

In [ ]:
role = sagemaker.get_execution_role()
session = boto3.session.Session()
region = session.region_name
s3 = session.resource("s3")
bucket = s3.Bucket(BUCKET_NAME)
smclient = session.client("sagemaker")

bucket_region = \
    session.client("s3").head_bucket(Bucket=BUCKET_NAME)["ResponseMetadata"]["HTTPHeaders"]["x-amz-bucket-region"]
assert (
    bucket_region == region
), f"Your S3 bucket {BUCKET_NAME} and this notebook need to be in the same region."

if (region != "us-east-1"):
    print("WARNING: Rekognition Custom Labels functionality is only available in us-east-1 at launch")
    
# Initialise some empty variables we need to exist:
predictor_std = None
predictor_hpo = None

## Step 1: Review our algorithm details

We'll use the GluonCV (a deep learning framework built on MXNet) implementation of YOLOv3, and run it on top of the SageMaker-provided MXNet container.

As detailed in the [SageMaker Python SDK docs](https://sagemaker.readthedocs.io/en/stable/using_mxnet.html), our job is to implement a Python file (or bundle of files with a designated entry point) that:

* When run as a script, performs model training and saves the resultant model artifacts
* When imported as a module, defines functions which the framework server application can call to load the model; perform inference; and deserialize/serialize data from and to the web.

In cases like this one where extra dependencies (or newer versions) are required vs the base container, there are two options:

* Define a custom container, and take on the effort of re-implementing (or inheriting) the framework server application code
* Performing a `pip install` in the code itself, executed when the file is loaded.

The latter option increases the billable execution time in training, and the latency for new container instances to spin-up in deployed endpoint auto-scaling... But for a small number of additional packages these costs can be preferable versus the complexity of fully customizing the container.

Take some time to look through our implementation at the location below in this repository:

In [ ]:
entry_point="yolo_train.py"
source_dir="src"

As with built-in algorithms, the choices we make in implementation will have consequences including for example:

* Whether distributed training is supported
* Whether GPU-accelerated instances will provide any performance benefits
* What data formats are supported for training and inference
* How data is loaded into the container at training time

## Step 2: Set up input data channels

From a script complexity perspective, the simplest approach would be to choose `s3_data_type="S3Prefix"` and estimator `input_mode="File"` (the defaults): in which case the matching files in S3 are simply downloaded to the container before the training job starts, and our training job code should load those to an in-memory dataset. This is the approach taken in the [official SageMaker MXNet MNIST example](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker-python-sdk/mxnet_mnist).

File mode is easy to use, but for jobs with large data sets we can cut container disk size and job startup time requirements by using **Pipe Mode** instead. Instead of downloading the full dataset to your container before starting work, Pipe Mode sets up a **stream** - which looks like a file on disk but only supports sequential access. Because the stream can't seek back to the start, SageMaker creates a new stream for each epoch - for example `/opt/ml/input/train/train_0` for the first.

We'd still like SageMaker to shuffle our data, so how do we handle a use case like this where we need *both* the image file and its corresponding annotations to be useful?

One solution could be to provide two separate channels for the images and the annotations (like the SageMaker built-in Image Classification algorithm when training with ["Image Format"](https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html)).

A more feature-rich approach is the **AugmentedManifestFile** data type we used in the previous example. With this method, SageMaker steps through the manifest file and produces *multiple records in the stream for each row*: One for each (potentially filtered) attribute.

Attribute names ending `-ref` (like our `source-ref`) are resolved to the raw contents of the S3 file URI they point to. Other attributes are passed through as plain text (i.e. JSON)... So in our case, the file stream in the container receives alternating records of image data and JSON annotations.

As you might imagine (and will see in the training script file), using Pipe Mode and AugmentedManifestFile means we have to translate this stream into the (probably different) format our models expect. This is easier for some libraries than others (for e.g. there's a [PipeModeDataset](https://github.com/aws/sagemaker-tensorflow-extensions#sagemaker-pipe-mode) class built-in for TensorFlow users), and Gluon makes things particularly difficult by implementing random access as a standard interface of base dataset classes: making them unsuitable for streaming in general.

Here we'll use the **same AugmentedManifestFile channel configuration as the built-in algorithm** from the previous notebook; and stream batches of data in to GluonCV datasets to train one after the other.

In [ ]:
train_channel = sagemaker.session.s3_input(
    f"s3://{BUCKET_NAME}/{DATA_PREFIX}/train.manifest",
    distribution="FullyReplicated",  # In case we want to try distributed training
    content_type="application/x-recordio",
    s3_data_type="AugmentedManifestFile",
    record_wrapping="RecordIO",
    attribute_names=attribute_names,  # In case the manifest contains other junk to ignore (it does!)
    shuffle_config=sagemaker.session.ShuffleConfig(seed=1337)
)
                                        
validation_channel = sagemaker.session.s3_input(
    f"s3://{BUCKET_NAME}/{DATA_PREFIX}/validation.manifest",
    distribution="FullyReplicated",
    content_type="application/x-recordio",
    record_wrapping="RecordIO",
    s3_data_type="AugmentedManifestFile",
    attribute_names=attribute_names
)

## Step 3: Configure the algorithm

The core configuration process for the custom algorithm is the same as for the built-in algorithm (such as instance type/count, data source, execution role, and so on)... and we've written this script to accept training and inference data in the same format as the built-in.

There are some differences too though, and some of the most significant ones are as follows:

**Need to define metrics:** Metrics in SageMaker (like `validation:mAP` which we see in the SageMaker console and used to tune the built-in algorithm) are captured from the console output of the algorithm via a [regular expression](https://www.regular-expressions.info/). For the built-in algorithms, SageMaker already knows what metrics are defined... But for a custom algorithm (since we could be `print()`ing whatever we like), we need to supply the definitions.

**No explicit "training_image"...:** As discussed in the last notebook, we only needed to use the generic `Estimator` SDK and supply a Docker image URI for the built-in SSD algorithm because there wasn't a specific SDK class for it. MXNet [has one](https://sagemaker.readthedocs.io/en/stable/using_mxnet.html) though, so the image URI is implicit. You can find more details about the (actually separate) SageMaker MXNet [training](https://github.com/aws/sagemaker-mxnet-container) and [inference](https://github.com/aws/sagemaker-mxnet-serving-container) containers in GitHub, as well as the shared [cross-framework base](https://github.com/aws/sagemaker-containers) they inherit from.

**...But need to specify framework versions:** Because multiple incompatible versions of Python (2 vs 3), or the ML framework (e.g. TensorFlow 1 vs 2) may be supported at a given time.

**Custom hyperparameters:** As with metrics, it's entirely up to us what hyperparameters we want to support and how: So any hyperparameters are fully custom to our code in the [src](src) folder. There's also a little inconsistency between frameworks and built-in algorithms about whether hyperparameters are passed in as a `hyperparameters` argument in the constructor or with a separate `set_hyperparameters()` function, but otherwise the process is the same.

In [ ]:
metric_definitions = [
    # You should see these metrics appear in the SageMaker console on the details page for the
    # training job when it starts (and see them in the log stream during `.fit()`):
    { "Name": "train:TrainingCost", "Regex": r"\[Epoch \d+\] .*TrainingCost=(.*?);" },
    { "Name": "train:ObjLoss", "Regex": r"\[Epoch \d+\] .*ObjLoss=(.*?);" },
    { "Name": "train:BoxCenterLoss", "Regex": r"\[Epoch \d+\] .*BoxCenterLoss=(.*?);" },
    { "Name": "train:BoxScaleLoss", "Regex": r"\[Epoch \d+\] .*BoxScaleLoss=(.*?);" },
    { "Name": "train:ClassLoss", "Regex": r"\[Epoch \d+\] .*ClassLoss=(.*?);" },
    { "Name": "train:TotalLoss", "Regex": r"\[Epoch \d+\] .*TotalLoss=(.*?);" },
    { "Name": "train:MeanAP", "Regex": r"Train: VOCMeanAP=(.*?);" },
    { "Name": "validation:MeanAP", "Regex": r"Validation: VOCMeanAP=(.*?);" },
]

# Review which of these parameters correspond to the built-in notebook, and which are new:
estimator = SageMakerMXNet(
    role=role,
    entry_point=entry_point,
    source_dir=source_dir,
    framework_version="1.4.1",
    py_version="py3",
    input_mode="Pipe",
    train_instance_count=1,  # Note: Our implementation doesn't actually support multi-instance yet
    train_instance_type="ml.p3.2xlarge",
    train_volume_size=50,
    train_max_run=60*60,
    train_use_spot_instances=True,
    train_max_wait=60*60,
    metric_definitions=metric_definitions,
    base_job_name="bootsncats-yolo",
    output_path=f"s3://{BUCKET_NAME}/{MODELS_PREFIX}",
    checkpoint_s3_uri=f"s3://{BUCKET_NAME}/{CHECKPOINTS_PREFIX}",
    # There's more documentation of the hyperparameters in the script itself:
    hyperparameters={
        "epochs": 200,
        "learning-rate": 0.0001,
        "num-classes": 2,
        #"num-workers": 1,
        "batch-size": 4,
        "data-shape": 416,
        "random-seed": 1337,
        "log-interval": 1,  # Log intermediate metrics every minibatch
        "early-stopping": "yes",
        #"log-level": "DEBUG",
    },
)

## Step 4: Train the model

As with the built-in algorithms, we have the choice between fitting our model with the given set of hyperparameters or performing automatic hyperparameter tuning:

In [ ]:
WITH_HPO = False# TODO: True first, then False?

In [ ]:
%%time
if (not WITH_HPO):
    estimator.fit(
        {
            "train": train_channel,
            "validation": validation_channel
        },
        logs=True
    )
else:
    hyperparameter_ranges = {
        "learning-rate": sagemaker.tuner.ContinuousParameter(0.00001, 0.01),
        "momentum": sagemaker.tuner.ContinuousParameter(0.0, 0.99),
        "weight_decay": sagemaker.tuner.ContinuousParameter(0.0, 0.99),
        "batch-size": sagemaker.tuner.IntegerParameter(1, 4),
        "optimizer": sagemaker.tuner.CategoricalParameter(["sgd", "adam", "rmsprop", "adadelta"])
    }

    tuner = sagemaker.tuner.HyperparameterTuner(
        estimator,
        "validation:MeanAP",  # Name of the objective metric to optimize
        objective_type="Maximize",  # "Mean Average Precision" high = good
        metric_definitions=metric_definitions,
        hyperparameter_ranges=hyperparameter_ranges,
        base_tuning_job_name="bootsncats-ssd-hpo",
        # `max_jobs` obviously has cost implications, but the optimization can always be terminated:
        max_jobs=24,
        max_parallel_jobs=3  # Keep sensible for the configured max_jobs...
    )
    
    tuner.fit(
        {
            "train": train_channel,
            "validation": validation_channel
        },
        include_cls_metadata=False
    )

Note that, if we ever lose notebook state e.g. due to a kernel restart or crash, we can `attach()` our estimator/tuner to a previous training/tuning job as follows: (No need to re-train - the results are all stored!)

In [ ]:
# Examples to attach to a previous training run:
#estimator.attach("bootsncats-ssd-hpo-191209-1637-003-bbdca4b2")
#tuner.attach("bootsncats-ssd-hpo-191209-1637")
#WITH_HPO=?

## Step 5: While the model(s) are training

Remember to go back to the previous notebooks if you still have steps unfinished...

Otherwise, take some time to read through and understand the code in the src folder! This implementation is made unusually complex, because of the nature of object detection data (image + bounding boxes) and our decision to support the same I/O as the built-in algorithm.

## Step 6: Deploy the model

Deploying our trained custom model is as simple as for the built-in algorithm, with just one extra step:

We need to warn the `predictor` our request body will be an image, so it passes this information along to our container.

In [ ]:
%%time
if (WITH_HPO):
    if (predictor_hpo):
        try:
            predictor_hpo.delete_endpoint()
            print("Deleted previous HPO endpoint")
        except:
            print("Couldn't delete previous HPO endpoint")
    print("Deploying HPO model...")
    predictor_hpo = tuner.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.large",
        # wait=False,
    )
    predictor_hpo.content_type = "application/x-image"
else:
    if (predictor_std):
        try:
            predictor_std.delete_endpoint()
            print("Deleted previous non-HPO endpoint")
        except:
            print("Couldn't delete previous non-HPO endpoint")
    print("Deploying standard (non-HPO) model...")
    predictor_std = estimator.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.large",
        # wait=False,
    )
    predictor_std.content_type = "application/x-image"

## Step 7: Run inference on test images

We've set our custom algorithm up like the built-in, to return *all* detections and let the caller filter by whatever confidence threshold seems to perform best.

Training this algorithm can be even trickier than SSD though: are you able to get good results?

In [ ]:
# Change this if you want something different:
predictor = predictor_hpo if WITH_HPO else predictor_std

# This time confidence is 0-1, not 0-100:
confidence_threshold = # TODO: 0.2 is a good starting point, but explore options!

for test_image in os.listdir(test_image_folder):
    test_image_path = f"{test_image_folder}/{test_image}"
    with open(test_image_path, "rb") as f:
        payload = bytearray(f.read())

    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
        EndpointName=predictor.endpoint,
        ContentType='application/x-image',
        Body=payload
    )

    result = response['Body'].read()
    result = json.loads(result)["prediction"]
    print(result)
    # result is a list of [class_ix, confidence, x1, y1, x2, y2] detections.
    # (x/y locations are relative/normalized)
    display(HTML(f"<h4>{test_image}</h4>"))
    util.visualize_detection(
        test_image_path,
        result,
        CLASS_NAMES,
        thresh=confidence_threshold
    )

## Clean up

Although training instances are ephemeral, the resources we allocated for real-time endpoints need to be cleaned up to avoid ongoing charges.

The code below will delete the *most recently deployed* endpoint for the HPO and non-HPO configurations, but note that if you deployed either more than once, you might end up with extra endpoints.

To be safe, it's best to still check through the SageMaker console for any left-over resources when cleaning up.

In [ ]:
if (predictor_hpo):
    print("Deleting HPO-optimized predictor endpoint")
    predictor_hpo.delete_endpoint()
if (predictor_std):
    print("Deleting standard (non-HPO) predictor endpoint")
    predictor_std.delete_endpoint()

## Review TODO

**TODO: Review**